In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
id_information = pd.read_csv('id_information_mmsr.tsv', sep='\t')
id_genres = pd.read_csv('id_genres_mmsr.tsv', sep='\t')
id_bert = pd.read_csv('id_bert_mmsr.tsv', sep='\t')
id_lyrics_tfidf = pd.read_csv('id_lyrics_tf-idf_mmsr.tsv', sep='\t')
id_lyrics_word2vec = pd.read_csv('id_lyrics_word2vec_mmsr.tsv', sep='\t')

In [ ]:
id_lyrics_tfidf['id']

In [ ]:
artist = 'Dua Lipa'
song_query = 'New Rules (Initial Talk Remix)'
song = id_information.loc[(id_information['artist']==artist)&(id_information['song']==song_query)]

In [ ]:
query_id = song.iloc[0]['id']

In [ ]:
tfidf_query = id_lyrics_tfidf.loc[id_lyrics_tfidf['id']==song.iloc[0]['id']]
tfidf_query = np.array(tfidf_query.drop(columns=['id']))

In [ ]:
tfidf_subset = id_lyrics_tfidf#.head(10000)
tfidf_subset

In [ ]:
#def cosine_similarity(x,y):
#    return np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))

In [ ]:
subset_ids = tfidf_subset[['id']].copy()
subset_ids

In [ ]:
tfidf_no_id = np.array(tfidf_subset.drop(columns=['id']))
cosine = cosine_similarity(tfidf_query, tfidf_no_id)
subset_ids.insert(1,'cosine', cosine[0])
subset_ids

In [ ]:
subset_ids= subset_ids.sort_values('cosine', ascending=False)

In [ ]:
sorted_cosine = subset_ids.iloc[0: , :]
sorted_cosine

In [ ]:
top_10 = sorted_cosine.head(10)
top_10

In [ ]:
top_10_ids = np.array(top_10['id'])

In [ ]:
for idx in top_10_ids:
    print(id_information.loc[(id_information['id'] == idx)][['artist', 'song']])

In [ ]:
top_10_meta = id_information.loc[id_information['id'].isin(top_10_ids)]  # not ordered

In [ ]:
query_genre = id_genres.loc[id_genres['id']==query_id]['genre']
query_genre

In [ ]:
def get_genre(song_id):
    genres = id_genres.loc[id_genres['id']==song_id]['genre'].tolist()[0]
    return genres

In [ ]:
def string_to_list(string):
    lst = string[1:-1].split(', ')
    return lst

In [ ]:
query_genre = get_genre(query_id)
print(query_genre)
query_genre = string_to_list(query_genre)

In [ ]:

tp = 0
for idx in top_10_ids:
    genres = get_genre(idx)
    genre_lst = string_to_list(genres)[0]
    if any(item in query_genre[0] for item in genre_lst):
        tp += 1

precision = tp/len(top_10_ids)
print(precision)

In [3]:
def get_genre(song_id):
    genres = id_genres.loc[id_genres['id']==song_id]['genre'].tolist()[0]
    return genres

def string_to_list(string):
    lst = string[1:-1].split(', ')
    return lst

def get_top_k(artist, song_query, k:int):
    song = id_information.loc[(id_information['artist']==artist)&(id_information['song']==song_query)]
    query_id = song.iloc[0]['id']
    tfidf_query = id_lyrics_tfidf.loc[id_lyrics_tfidf['id']==song.iloc[0]['id']]
    tfidf_query = np.array(tfidf_query.drop(columns=['id']))
    tfidf_ids = id_lyrics_tfidf[['id']].copy()
    tfidf_no_id = np.array(id_lyrics_tfidf.copy().drop(columns=['id']))
    cosine = cosine_similarity(tfidf_query, tfidf_no_id)
    tfidf_ids.insert(1,'cosine', cosine[0])
    tfidf_ids= tfidf_ids.sort_values('cosine', ascending=False)
    sorted_cosine = tfidf_ids.iloc[1: , :]
    top_k = sorted_cosine.head(k)
    top_k_ids = np.array(top_k['id'])
    query_genre = get_genre(query_id)
    query_genre = string_to_list(query_genre)
    
    tp = 0
    for idx in top_k_ids:
        genres = get_genre(idx)
        genre_lst = string_to_list(genres)[0]
        if any(item in query_genre[0] for item in genre_lst):
            tp += 1
    precision = tp/len(top_k_ids)  # somethings wrong
    
    for idx in top_k_ids:
        print(id_information.loc[(id_information['id'] == idx)][['artist', 'song']])
    
    return precision

In [4]:
print(get_top_k('Dua Lipa', 'New Rules (Initial Talk Remix)', 10))

         artist                            song
58942  Dua Lipa  New Rules - Initial Talk Remix
         artist       song
52034  Dua Lipa  New Rules
              artist                           song
39323  Lenny Kravitz  Let Love Rule (Justice Remix)
        artist song
42582  J. Cole  ATM
                   artist         song
50654  Jefferson Starship  Count on Me
                artist         song
17479  Whitney Houston  Count On Me
          artist         song
2424  Bruno Mars  Count on Me
        artist      song
44737  Ramones  Commando
     artist    song
2019   Kllo  Virtue
                  artist                           song
73725  The Dandy Warhols  Every Day Should Be A Holiday
1.0
